In [3]:
import os, sys

import torch
import scipy.stats as st

In [4]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(p))
    if module_path not in sys.path:
        sys.path.append(module_path)

models_path = '../models'

In [5]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset)

train = train.to_sequence()
test = test.to_sequence()

In [6]:
# load trained model
ofile = 'entire_model_1m.pt'
model = torch.load(os.path.join(models_path, ofile))

In [7]:
# initialize input parameters
k = 10
no_interactions = 5
user_id = 8

In [14]:
class ItemInteractionInfo:
    def __init__(self, uid, iid, p=-1, i=None):
        self.user_id = uid
        self.item_id = iid
        self.pos = p
        self.interactions = i
        
    def __str__(self): 
        items_order = [(n[0], n[1].detach().numpy().flatten()[0]) if isinstance(n[1], torch.Tensor) else (n[0], n[1]) for n in self.items_order]
            
        return (f'user_id: {self.user_id}, item_id: {self.item_id}\n'
                f'Found in iter {self.counter_found_best} with score/in pos {self.score} with interactions {self.interactions}\n'
                f'10-best proposed items {items_order}')
    
    user_id = -1
    item_id = -1
    score = 0
    pos = -1
    interactions = []
    items_order = []
    counter_found_best = -1

In [20]:
items_interacted = test.sequences[test.user_ids==user_id][0]
predictions = -model.predict(items_interacted[:no_interactions])

print(f'Given the following interactions {items_interacted[:no_interactions]} for user {user_id} the next most {k} possible items'
      f'to interact with are {list(predictions.argsort()[:k])}')
cand = input('Choose one of the above next interacted items that should become less candidate: ')
try:
    cand = int(cand)
except ValueError:
    print("That's not an int!")

print(f'Current pos of selected item {cand} is {st.rankdata(predictions, method="ordinal")[cand]}\n')

Given the following interactions [446 508 447 488 472] for user 8 the next most 10 possible itemsto interact with are [482, 28, 582, 473, 641, 499, 1025, 1409, 1385, 1593]


Choose one of the above next interacted items that should become less candidate:  28


Current pos of selected item 28 is 2



In [25]:
from itertools import permutations
import torch.nn.functional as F


counter = 1
best_inter = ItemInteractionInfo(user_id, cand)

for l in range(1, no_interactions + 1):
#     produce permutations of various interactions
    perm = permutations(items_interacted[:no_interactions], l)    

    for i in list(perm):
#         predict next top-k items about to be selected
        preds = model.predict(i) 
#     convert logits produced by model, i.e., the probability distribution before normalization, by using softmax
        tensor = torch.from_numpy(preds).float()
        preds = F.softmax(tensor, dim=0)
#         get the ranking position of selected item in the list
        item_pos = st.rankdata(-preds, method='ordinal')[cand]
#     keep info about the best solution found depending on an objective function
        if item_pos > best_inter.score:
            best_inter.score = item_pos
            best_inter.interactions = i
            best_inter.items_order = sorted(enumerate(preds), key=lambda x: x[1], reverse=True)[:k]
            best_inter.counter_found_best = counter

        counter += 1
    
print(best_inter, f'\nTotal iterations: {counter}')

user_id: 8, item_id: 28
Found in iter 53 with score/in pos 43 with interactions (447, 508, 446)
10-best proposed items [(462, 0.0033340384), (485, 0.003119285), (461, 0.002942569), (60, 0.002781871), (466, 0.002767015), (449, 0.002645208), (57, 0.0025820122), (313, 0.0025489107), (686, 0.0022733852), (927, 0.0022616663)] 
Total iterations: 326
